### Sample notebook to show how to integrate litellm in python

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import os
import json
from pprint import pprint

In [7]:
import boto3
import os
import hmac
import base64
import hashlib
from botocore.exceptions import ClientError

# Set environment variables

os.environ['COGNITO_CLIENT_SECRET'] = '11181idr8ho95i6k67v99p3go2sn72kri5j5qdg2lo4jl0uj9ri4'
os.environ['COGNITO_PASSWORD'] ='xxxxxxxxxxx'
os.environ['COGNITO_USERNAME'] ='xxxxxxxxxxx'
os.environ['COGNITO_USER_POOL_ID'] ='us-east-1_7Rq8X1q2q'
os.environ['COGNITO_CLIENT_ID'] ='7v15em5a0iqvb3hn5r69cg3485'

def calculate_secret_hash(username, client_id, client_secret):
    message = username + client_id
    dig = hmac.new(
        key=client_secret.encode('utf-8'),
        msg=message.encode('utf-8'),
        digestmod=hashlib.sha256
    ).digest()
    return base64.b64encode(dig).decode()

def get_cognito_token():
    try:
        # Initialize Cognito Identity Provider client
        client = boto3.client('cognito-idp', region_name='us-east-1')
        
        # Get credentials from environment variables
        client_id = os.environ.get('COGNITO_CLIENT_ID')
        client_secret = os.environ.get('COGNITO_CLIENT_SECRET')
        username = os.environ.get('COGNITO_USERNAME')
        password = os.environ.get('COGNITO_PASSWORD')
        
        # Calculate secret hash
        secret_hash = calculate_secret_hash(username, client_id, client_secret)
        
        # Initiate auth flow
        response = client.initiate_auth(
            ClientId=client_id,
            AuthFlow='USER_PASSWORD_AUTH',
            AuthParameters={
                'USERNAME': username,
                'PASSWORD': password,
                'SECRET_HASH': secret_hash  # Use the calculated secret hash here
            }
        )
        
        return {
            "status": 200,
            "access_token": response['AuthenticationResult']['AccessToken']
        }
        
    except ClientError as e:
        return {
            "status": 500,
            "message": str(e)
        }

# Usage
response = get_cognito_token()
if response["status"] == 200:
    token = response["access_token"]
    print(token)
else:
    print(f"Error getting access token: {response['message']}")

eyJraWQiOiJpb2dUeXR5amt4WFVtcWM5TFVEVmN4NEx3VkY4WEtnN0RzRlwvUU1yMngzWT0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiIxNGU4NzRiOC1jMGUxLTcwNzQtOTQwNC1hMWRhYTM5NWYxODMiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV82cU4zY0tGZWIiLCJjbGllbnRfaWQiOiI3djE1ZW01YTBpcXZiM2huNXI2OWNnMzQ4NSIsIm9yaWdpbl9qdGkiOiI3MDFkODFkNS1mM2YxLTRmYTEtYmRmZi04OWMyMDRiNjU2NGEiLCJldmVudF9pZCI6IjIxOTA5YWQ2LThlMGMtNGZhNi04YTFiLWM5NzBiZGUyYTFhYiIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3MzE2ODU3NzEsImV4cCI6MTczMTY4OTM3MSwiaWF0IjoxNzMxNjg1NzcxLCJqdGkiOiI3ODZkOTgzZC03YTBlLTQ0NjItOWQwZS0wMGYyMWY3YjgyMzAiLCJ1c2VybmFtZSI6IjE0ZTg3NGI4LWMwZTEtNzA3NC05NDA0LWExZGFhMzk1ZjE4MyJ9.EXMihaIgHoT9vo8r7hML5PaHiYELKqAHExFd3kZl0p3tWJrRU5ZPlotU_nxUREmGvrPAs1j60uGYQYaQH1v0J2f-x_17KDEcGdX8LEpw2yXvzSDHv2_E6qEzgcampRWrzIndpaIWDHznzV-usydiL61UUkaVs0DeenXqCRTsGvG_Yov-pjmB9_1a2SP0t5QYaFs1eEtqW_PJlKCPHChg6vYPo9iBGwkatmvcuk9Zt3YTGHBFAAPx3N9_OVF1leXu5mJ8V7jVMjU8aX

#### Setup
* We will be using litellm proxy to connect to any LLM of your choice
* Some LLMs can be used for embedding and other for inference

In [8]:
# this is URL will vary by DEPLOYMENT ENVIRONMENT
#litellm_proxy_endpoint = os.environ.get(
#    "litellm_proxy_endpoint",
#    "https://llm-proxy-client-tech-labs-dev.mesh.eks-ifs-dev-east-01.aws.tiaa-cref.org")

litellm_proxy_endpoint = os.environ.get(
    "litellm_proxy_endpoint",
    "https://api-llm.ctl-gait.clientlabsaft.com")
temperature = 0
max_tokens = 4096
bearer_token=get_cognito_token()['access_token']
#bearer_token = 'eyJraWQiOiJpb2dUeXR5amt4WFVtcWM5TFVEVmN4NEx3VkY4WEtnN0RzRlwvUU1yMngzWT0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiIxNGU4NzRiOC1jMGUxLTcwNzQtOTQwNC1hMWRhYTM5NWYxODMiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV82cU4zY0tGZWIiLCJ2ZXJzaW9uIjoyLCJjbGllbnRfaWQiOiI3djE1ZW01YTBpcXZiM2huNXI2OWNnMzQ4NSIsIm9yaWdpbl9qdGkiOiJkOTZlZTc2MC0zZWQ3LTQ0YmUtOTc1MS03ZGIyZTZkMWU2N2UiLCJldmVudF9pZCI6IjI2NTk5Y2Q4LWVlYTAtNDc5Yi05MmFiLTdhZDVlMjU1MzM1YyIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoib3BlbmlkIiwiYXV0aF90aW1lIjoxNzMxNjgwNDA1LCJleHAiOjE3MzE2ODQwMDUsImlhdCI6MTczMTY4MDQwNSwianRpIjoiMTJlMDJjYjgtNDYwYy00NGIyLWI4YzgtMzFlZGYzMDE2Nzk2IiwidXNlcm5hbWUiOiIxNGU4NzRiOC1jMGUxLTcwNzQtOTQwNC1hMWRhYTM5NWYxODMifQ.mTcQ3G8wQnUIJCnWqC1NZ9dXtX4IoftQALjAWAMJqSo-2ovG2Zf-4Jq81ABQZzXZHxgkSWLLSI4Ccq96Nr4tTWpCsUihzkTMkMah6BLRqoYf1QYwZUNGH1Fu8vqZMZNTQRDt3BV12rn3f_d8gvQJ0UEKUOW59FZ3yoEr6VdKnClW2BGiDmQxoBO8EQGgNSqi6ectXif7LdG_dc8-KPoq42a3tJKZsB4FkcICHxIucFZMjqHFIqWoSrr2LhfS5WBko3A3WaCqAyEOIVDHV2KiGY4nr_dlfbYHi5EMmI3BgSxZz29qGNSyzJr9mHn5VOE7jiDpFE1-ydrePw6sDoVTIg'
x_api_key = 'xxxxxxxxxxxxxxxx' 
# This will be provided for each team/use case/project
 


#### Discover all models (inference and embedding) - run the request in next cell
* mode=_**chat**_ - this should be used for inference
* mode=_**emebedding**_ - this should be used for embedding

In [9]:
import requests

url = f"{litellm_proxy_endpoint}/model/info"

payload = ""
headers = {
  'Authorization': f'Bearer {bearer_token}',
  'x-api-key': 'xxxxxxxxxxx' 
}

response = requests.request("GET", url, headers=headers, data=payload, verify=True)#'cacert.pem')

pprint(json.loads(response.text))



{'data': [{'litellm_params': {'model': 'vertex_ai/claude-3-5-sonnet@20240620',
                              'vertex_location': 'us-east5',
                              'vertex_project': 'clabs-int-ctl-gait-llm-082824'},
           'model_info': {'cache_creation_input_token_cost': None,
                          'cache_read_input_token_cost': None,
                          'db_model': False,
                          'id': 'e5b42f125b454405af0c6f111234179787549efaf139e6e7d0c425d53bc91bd0',
                          'input_cost_per_audio_token': None,
                          'input_cost_per_character': None,
                          'input_cost_per_query': None,
                          'input_cost_per_second': None,
                          'input_cost_per_token': 3e-06,
                          'input_cost_per_token_above_128k_tokens': None,
                          'key': 'vertex_ai/claude-3-5-sonnet@20240620',
                          'litellm_provider': 'vertex_ai-anthrop

#### we will use OpenAI chat to talk to any LLMs (even if they are not OpenAI LLM)
* We will first test non-streaming
* this will impact gAIt Streaming and Ingestion containers (since you make LLM calls in ingestion too)

In [10]:
CHOSEN_LITE_LLM_MODEL = 'Anthropic Claude-V3.5 Sonnet Vertex AI (Internal)'


chat = ChatOpenAI(
    openai_api_base=litellm_proxy_endpoint, # set openai_api_base to the LiteLLM Proxy
    model = CHOSEN_LITE_LLM_MODEL,
    default_headers={'x-api-key': 'Bearer sk-isbnmolh-BLL-CYaTZzp6w'},
    temperature=temperature,
    api_key=bearer_token,
    streaming=False,
    #max_tokens=max_tokens,
    user=bearer_token
)

/var/folders/tp/vml9088d54s9lvyb6st_qp89_vfkd4/T/ipykernel_53238/2896859195.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(
WARNING! user is not default parameter.
                    user was transferred to model_kwargs.
                    Please confirm that user is what you intended.


In [11]:
chat.invoke(['how are you?', 'who are you?', 'who is martin luther king?'])

AIMessage(content='I\'m doing well, thank you for asking. I am an AI language model created by OpenAI, designed to assist with information and answer questions to the best of my ability based on my training data.\n\nAs for Martin Luther King Jr., he was a prominent American civil rights leader and Baptist minister who played a crucial role in the advancement of civil rights in the United States through nonviolent resistance. Here are some key points about him:\n\n1. Born on January 15, 1929, in Atlanta, Georgia.\n\n2. He became a leader in the African-American civil rights movement of the 1950s and 1960s.\n\n3. Known for his powerful speeches, including the famous "I Have a Dream" speech delivered in 1963 during the March on Washington.\n\n4. Advocated for racial equality, voting rights, and an end to segregation through peaceful protest and civil disobedience.\n\n5. Received the Nobel Peace Prize in 1964 for his efforts to end racial discrimination.\n\n6. Assassinated on April 4, 1968

In [13]:
some_content = """
Write me essay about TIAA in 50 words?
"""

messages = [
    SystemMessage(content="You are a instruction-tuned large language model. Follow the user's instructions carefully. Respond using markdown."),
    #SystemMessage(content="Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity."),
    HumanMessage(content=some_content),
]

try:
    response = chat(messages)
    print(response)
except Exception as e:
    json_str = str(e).split(" - ", 1)[1]
    print(eval(json_str)['error']['message'])

content="Here's a 50-word essay about TIAA in markdown format:\n\n# TIAA: Securing Financial Futures\n\nTIAA, founded in 1918, is a leading provider of financial services for those in academic, research, medical, and cultural fields. Offering retirement plans, insurance, and investment solutions, TIAA has helped millions achieve financial security. With a century-long commitment to social responsibility and ethical investing, TIAA continues to empower individuals to build a secure financial future." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 45, 'total_tokens': 164, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0}, 'model_name': 'Anthropic Claude-V3.5 Sonnet Vertex AI (Internal)', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-dd7c18e3-44b7-45d6-9652-72f3b92ddde3-0'


#### Lets now test streaming

In [14]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [15]:
chat = ChatOpenAI(
    openai_api_base=litellm_proxy_endpoint, # set openai_api_base to the LiteLLM Proxy
    model = CHOSEN_LITE_LLM_MODEL,
    temperature=temperature,
    api_key=bearer_token,
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    #max_tokens=max_tokens,
#     user=bearer_token,
    default_headers={'x-api-key': 'xxxxxxxxxxxxx'},
    model_kwargs = {'user': bearer_token}
    
)


chat(messages)


/var/folders/tp/vml9088d54s9lvyb6st_qp89_vfkd4/T/ipykernel_53238/537323102.py:1: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  chat = ChatOpenAI(


Here's a 50-word essay about TIAA in markdown format:

# TIAA: Securing Financial Futures

TIAA, founded in 1918, is a leading provider of financial services for those in academic, research, medical, and cultural fields. Offering retirement plans, insurance, and investment solutions, TIAA has helped millions achieve financial security. With a century-long commitment to social responsibility and ethical investing, TIAA continues to empower individuals to build a secure financial future.

AIMessage(content="Here's a 50-word essay about TIAA in markdown format:\n\n# TIAA: Securing Financial Futures\n\nTIAA, founded in 1918, is a leading provider of financial services for those in academic, research, medical, and cultural fields. Offering retirement plans, insurance, and investment solutions, TIAA has helped millions achieve financial security. With a century-long commitment to social responsibility and ethical investing, TIAA continues to empower individuals to build a secure financial future.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-c4686990-d52b-43b3-8f9d-a44ffeabb705-0')

#### Lets now test embedding model
* remember the length of vector can vary based on dimensions
* use dimensions to set dimensions parameter in opensearchvector collection (when you create it)
* this will impact gAIt Streaming and Ingestion containers

In [16]:
from langchain.embeddings.openai import OpenAIEmbeddings

CHOSEN_LITE_LLM_EMBEDDING_MODEL = 'Gemini Gecko - Vertex AI Text Embedding (Internal)'

embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    model= CHOSEN_LITE_LLM_EMBEDDING_MODEL,
    openai_api_base=litellm_proxy_endpoint,
    openai_api_key=bearer_token,
    #chunk_size=256 # 5-16
    default_headers={'x-api-key': 'xxxxxxxxxxxxxxxx'},
    user=bearer_token
)

result = embeddings.embed_documents(['this is great', 'this is awesome'])
print(len(result))


/var/folders/tp/vml9088d54s9lvyb6st_qp89_vfkd4/T/ipykernel_53238/1840633978.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
/Users/ur0p3x4/anaconda3/lib/python3.10/site-packages/langchain_community/embeddings/openai.py:271: UserWarning: WARNING! user is not default parameter.
                    user was transferred to model_kwargs.
                    Please confirm that user is what you intended.
  warnings.warn(


2


In [17]:
result[0]

[-0.00403308074710555,
 -0.023798438089969343,
 -0.036467188531338096,
 -0.03786622307259747,
 0.028454725302058385,
 -0.053423140718965666,
 -0.019049899534883424,
 -0.015136666184890086,
 0.024640608082442842,
 0.0036986903506953752,
 0.05694710216078967,
 0.027737902173456605,
 -0.006577897755532266,
 -0.04409166414790341,
 0.046080874825934114,
 0.019028855343368116,
 0.033691650033043925,
 -0.015030030546533237,
 0.011007501697545427,
 0.015844614729452682,
 0.011896621583038478,
 0.02384419402593797,
 -0.008444130857318039,
 -0.014092948419101487,
 -0.01322506271516929,
 -0.012311652974882879,
 0.031789838501258695,
 -0.0644664089739086,
 -0.021099577338879915,
 0.0028873376283667534,
 -0.05030142173121441,
 0.016751696124839708,
 -0.04070154137992795,
 -0.013180278149874696,
 0.026168524792539064,
 -0.022871424503627696,
 -0.011586604394897526,
 0.020629936847474016,
 0.017505714461475582,
 0.02707597312948464,
 0.02367809429729696,
 -0.06299564015192365,
 0.02453170653339575,
 

### Installation

* This tutorial requires these langchain dependencies:

In [2]:
!pip install langchain langchain_community langchain_chroma
!pip install langchainhub
!pip install -qU langchain-openai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 165.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 194.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 141.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 125.3 MB/s eta 0:00:00
  Creat

In [3]:
!pip install --no-deps langchain
!pip install --no-deps langchain-community
!pip install --no-deps langchain-core>=0.3.17
!pip install chromadb
!pip install langchainhub
!pip install langchain-openai


### Preview RAG

* In this guide we’ll build a QA app over as website. The specific website we will use is the LLM Powered Autonomous Agents blog post by Lilian Weng, which allows us to ask questions about the contents of the post.

* We can create a simple indexing pipeline and RAG chain to do this in ~20 lines of code:

In [18]:
CHOSEN_LITE_LLM_MODEL = 'Anthropic Claude-V3.5 Sonnet Vertex AI (Internal)'

llm = ChatOpenAI(
    openai_api_base=litellm_proxy_endpoint, # set openai_api_base to the LiteLLM Proxy
    model = CHOSEN_LITE_LLM_MODEL,
    temperature=temperature,
    api_key=bearer_token,
    streaming=False,
    #max_tokens=max_tokens,
    default_headers={'x-api-key': 'xxxxxxxxx'},
    user=bearer_token
)

WARNING! user is not default parameter.
                    user was transferred to model_kwargs.
                    Please confirm that user is what you intended.


In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings

CHOSEN_LITE_LLM_EMBEDDING_MODEL = 'Gemini Gecko - Vertex AI Text Embedding (Internal)'

embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    model= CHOSEN_LITE_LLM_EMBEDDING_MODEL,
    openai_api_base=litellm_proxy_endpoint,
    openai_api_key=bearer_token,
    default_headers={'x-api-key': 'xxxxxxxx'},
    #chunk_size=256 # 5-16
    user=bearer_token
)


/Users/ur0p3x4/anaconda3/lib/python3.10/site-packages/langchain_community/embeddings/openai.py:271: UserWarning: WARNING! user is not default parameter.
                    user was transferred to model_kwargs.
                    Please confirm that user is what you intended.
  warnings.warn(


In [23]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/ur0p3x4/anaconda3/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'Task Decomposition is a technique used to break down complex tasks into smaller, more manageable steps. It is often implemented using methods like Chain of Thought (CoT) or Tree of Thoughts, which instruct language models to "think step by step" and explore multiple reasoning possibilities. Task decomposition can be performed by language models through simple prompting, task-specific instructions, or with human input.'

### Additional Materials
* https://python.langchain.com/v0.2/docs/tutorials/
* https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/#as-a-retriever
